# Predicting crime in San Francisco with machine learning (Kaggle 2015)

__Notice:__ `sklearn-pandas` should be checked out from latest master branch
~~~
$ pip2 install --upgrade git+https://github.com/paulgb/sklearn-pandas.git@master
~~~

## Requirements

In [25]:
import datetime
import gc
import zipfile

import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn as sk
from pandas.tseries.holiday import USFederalHolidayCalendar
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.kernel_approximation import RBFSampler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer
from sklearn_pandas import DataFrameMapper

In [2]:
%matplotlib inline

In [3]:
DATADIR = "./data/"

## Raw data

In [4]:
train, test = pd.DataFrame(), pd.DataFrame()

with zipfile.ZipFile(DATADIR + "train.csv.zip", "r") as zf:
        train = pd.read_csv(zf.open("train.csv"), parse_dates=['Dates'])
        
with zipfile.ZipFile(DATADIR + "test.csv.zip", "r") as zf:
        test = pd.read_csv(zf.open("test.csv"), parse_dates=['Dates'])

In [5]:
train.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [6]:
test.head()

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412


## View composing & feature engineering

In [7]:
train_view, test_view = pd.DataFrame(), pd.DataFrame()

train_view["category"] = train["Category"]

for (view, data) in [(train_view, train), (test_view, test)]:
    view["district"] = data["PdDistrict"]
    view["hour"] = data["Dates"].map(lambda x: x.hour)
    view["weekday"] = data["Dates"].map(lambda x: x.weekday())
    view["day"] = data["Dates"].map(lambda x: x.day)
    view["dayofyear"] = data["Dates"].map(lambda x: x.dayofyear)
    view["month"] = data["Dates"].map(lambda x: x.month)
    view["year"] = data["Dates"].map(lambda x: x.year)
    view["lon"] = data["X"]
    view["lat"] = data["Y"]
    view["address"] = data["Address"].map(lambda x: x.split(" ", 1)[1] if x.split(" ", 1)[0].isdigit() else x)
    view["corner"] = data["Address"].map(lambda x: "/" in x)

In [8]:
days_off = USFederalHolidayCalendar().holidays(start='2003-01-01', end='2015-05-31').to_pydatetime()

for (view, data) in [(train_view, train), (test_view, test)]:
    view["holiday"] = data["Dates"].map(lambda x: datetime.datetime(x.year,x.month,x.day) in days_off)
    view["workhour"] = data["Dates"].map(lambda x: x.hour in range(9,17))
    view["sunlight"] = data["Dates"].map(lambda x: x.hour in range(7,19))

In [9]:
train_view.head()

,category,district,hour,weekday,day,dayofyear,month,year,lon,lat,address,corner,holiday,workhour,sunlight
0,WARRANTS,NORTHERN,23,2,13,133,5,2015,-122.425892,37.774599,OAK ST / LAGUNA ST,True,False,False,False
1,OTHER OFFENSES,NORTHERN,23,2,13,133,5,2015,-122.425892,37.774599,OAK ST / LAGUNA ST,True,False,False,False
2,OTHER OFFENSES,NORTHERN,23,2,13,133,5,2015,-122.424363,37.800414,VANNESS AV / GREENWICH ST,True,False,False,False
3,LARCENY/THEFT,NORTHERN,23,2,13,133,5,2015,-122.426995,37.800873,Block of LOMBARD ST,False,False,False,False
4,LARCENY/THEFT,PARK,23,2,13,133,5,2015,-122.438738,37.771541,Block of BRODERICK ST,False,False,False,False


In [10]:
test_view.head()

,district,hour,weekday,day,dayofyear,month,year,lon,lat,address,corner,holiday,workhour,sunlight
0,BAYVIEW,23,6,10,130,5,2015,-122.399588,37.735051,Block of THOMAS AV,False,False,False,False
1,BAYVIEW,23,6,10,130,5,2015,-122.391523,37.732432,3RD ST / REVERE AV,True,False,False,False
2,NORTHERN,23,6,10,130,5,2015,-122.426002,37.792212,Block of GOUGH ST,False,False,False,False
3,INGLESIDE,23,6,10,130,5,2015,-122.437394,37.721412,Block of MISSION ST,False,False,False,False
4,INGLESIDE,23,6,10,130,5,2015,-122.437394,37.721412,Block of MISSION ST,False,False,False,False


## Garbage Collection

In [11]:
del train
del test
gc.collect()

775

In [12]:
target_mapper = DataFrameMapper([
    ("category", LabelEncoder()),
])

y = target_mapper.fit_transform(train_view.copy())

print "sample:", y[0]
print "shape:", y.shape

sample: [37]
shape: (878049, 1)


In [14]:
data_mapper = DataFrameMapper([
    ("district", LabelBinarizer()),
    ("hour", StandardScaler()),
    ("weekday", StandardScaler()),
    ("day", StandardScaler()),
    ("dayofyear", StandardScaler()),
    ("month", StandardScaler()),
    ("year", StandardScaler()),
    ("lon", StandardScaler()),
    ("lat", StandardScaler()),
    ("address", [LabelEncoder(), StandardScaler()]),
    ("corner", LabelEncoder()),
    ("holiday", LabelEncoder()),
    ("workhour", LabelEncoder()),
    ("sunlight", LabelEncoder()),
])

data_mapper.fit(pd.concat([train_view.copy(), test_view.copy()]))
X = data_mapper.transform(train_view.copy())
X_test = data_mapper.transform(test_view.copy())

print "sample:", X[0]
print "shape:", X.shape

sample: [ 0.          0.          0.          0.          1.          0.          0.
  0.          0.          0.          1.46403971 -0.50189796 -0.29567607
 -0.43165691 -0.39872395  1.73120351 -0.1055266   0.00711031  2.05544797
  1.          0.          0.          0.        ]
shape: (878049, 23)


## Draw samples

In [15]:
samples = np.random.permutation(np.arange(X.shape[0]))[:100000]
X_sample = X[samples]
y_sample = y[samples]

In [16]:
y_sample = np.reshape(y_sample, -1)
y = np.reshape(y, -1)

## Stochastic Gradient Descent (SGD)

### SGD with Kernel Approximation

RBF Kernel by Monte Carlo approximation of its Fourier transform

In [17]:
sgd_rbf = make_pipeline(RBFSampler(gamma=0.1, random_state=1), SGDClassifier())

#### Parameter Search

In [18]:
alpha_range = 10.0**-np.arange(1,7)
loss_function = ["hinge", "log", "modified_huber", "squared_hinge", "perceptron"]

params = dict(
    sgdclassifier__alpha=alpha_range,
    sgdclassifier__loss=loss_function
)

In [ ]:
%%time
grid = GridSearchCV(sgd_rbf, params, cv=10, scoring="accuracy", n_jobs=1)
grid.fit(X_sample, y_sample)
print "best score:", grid.best_score_
print "parameter:", grid.best_params_

#### Training

In [ ]:
%%time
rbf = RBFSampler(gamma=0.1, random_state=1)
rbf.fit(np.concatenate((X, X_test), axis=0))
X_rbf = rbf.transform(X)
X_test_rbf = rbf.transform(X_test)

In [ ]:
%%time
sgd = SGDClassifier(loss="log", alpha=0.001, n_iter=1000, n_jobs=-1)
sgd.fit(X_rbf, y)

#### Classification

In [ ]:
results = sgd_predict_proba(X_test_rbf)

## Random Forest Classifier

In [ ]:
%%time
rfc = RandomForestClassifier(max_depth=16,n_estimators=1024)
cross_val_score(rfc, X, y, cv=10, scoring="accuracy", n_jobs=-1).mean()